In [1]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
import json
from tqdm import tqdm

In [2]:
# Load pre-trained model and tokenizer
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [3]:
def predict_answer(para_steps, question):
    # Join the steps into a single text
    text = ' '.join(para_steps)
    
    # Prepare the inputs for the model
    inputs = tokenizer(text, question, return_tensors='pt')
    
    # Get the model's predictions
    outputs = model(**inputs)

    # The model returns the logits (predictions before activation function)
    # We take the argmax to get the most likely answer label
    preds = torch.argmax(outputs.logits, dim=1)
    
    # Convert tensor to integer
    pred = preds.item()

    # Map predicted label to corresponding text
    label_to_text = {0: 'more', 1: 'less', 2: 'no_effect'}
    predicted_answer = label_to_text[pred]
    
    return predicted_answer

In [ ]:
# Load your data
JSONL_PATH = '../datasets/wiqa-dataset-v2-october-2019/test.jsonl'
with open(JSONL_PATH, 'r') as json_file:
    data = list(json_file)

In [ ]:
for entry in tqdm(data):
    entry = json.loads(entry)
    question = entry['question']['stem']
    para_steps = entry['question']['para_steps']
    predicted_answer = predict_answer(para_steps, question)
    
    if predicted_answer == answer_label:
        result['correct'] += 1
    else:
        result['incorrect'] += 1

 47%|████▋     | 13903/29808 [24:53<28:57,  9.16it/s] 

In [ ]:
json_file